In [39]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap as Basemap
import geopandas as gpd
import numpy as np
import os
import glob

In [3]:
df_2016 = pd.read_csv('State-level Simulations/Data/st_full_dairy.csv')
df_2016 = df_2016[['nb_shp', 'org_st', 'dest_st']]
df_2016

,nb_shp,org_st,dest_st
0,1,AL,TN
1,2,AL,KY
2,3,AL,FL
3,3,AL,CO
4,3,AL,IA
...,...,...,...
1141,2,WY,MN
1142,3,WY,OK
1143,170,WY,CO
1144,2,WY,SD


In [6]:
df_2016['nb_shp_norm_2016'] = (df_2016['nb_shp'] - df_2016['nb_shp'].min()) / (df_2016['nb_shp'].max() - df_2016['nb_shp'].min())
df_2016.sort_values(by='nb_shp_norm_2016', ascending=False)

,nb_shp,org_st,dest_st,nb_shp_norm_2016
71,3137,CA,TX,1.000000
40,2333,AZ,CA,0.743622
1082,1316,WI,IL,0.419324
699,1181,NM,TX,0.376276
1084,1073,WI,IA,0.341837
...,...,...,...,...
609,1,NC,LA,0.000000
606,1,NC,MN,0.000000
601,1,NC,ID,0.000000
596,1,NC,CT,0.000000


In [45]:
directory_path = "data/USAMMv3_cattle_networks/dairy"

data = []
count = 0
for file_path in glob.glob(os.path.join(directory_path, "dairy_network_*.network")):
    count += 1
    if count %50 ==0:
        print(f'{count} files processed.')
    df_2009 = pd.read_csv(file_path, sep = "\t")
    df_2009.rename(columns = {"oStateAbbr": "org_st", "dStateAbbr": "dest_st"}, inplace=True)
    df_2009['nb_shp'] = 1
    df_2009 = df_2009.groupby(['org_st', 'dest_st'])['nb_shp'].sum().reset_index()
    df_2009 = df_2009.loc[df_2009.org_st != df_2009.dest_st]
    df_2009['nb_shp_norm_2009'] = (df_2009['nb_shp'] - df_2009['nb_shp'].min()) / (df_2009['nb_shp'].max() - df_2009['nb_shp'].min())
    df_merged = pd.merge(df_2009, df_2016, on=['org_st', 'dest_st'], how='outer')
    df_merged[['nb_shp_norm_2016', 'nb_shp_norm_2009']] = df_merged[['nb_shp_norm_2016', 'nb_shp_norm_2009']].fillna(0)
    df_merged = df_merged[['org_st', 'dest_st', 'nb_shp_norm_2009', 'nb_shp_norm_2016']]
    rmse = np.sqrt(np.mean((df_merged['nb_shp_norm_2016'] - df_merged['nb_shp_norm_2009'])**2))
    data.append((file_path, rmse))

50 files processed.
100 files processed.
150 files processed.
200 files processed.
250 files processed.
300 files processed.
350 files processed.
400 files processed.
450 files processed.
500 files processed.
550 files processed.
600 files processed.
650 files processed.
700 files processed.
750 files processed.
800 files processed.
850 files processed.
900 files processed.
950 files processed.
1000 files processed.


In [59]:
df_results = pd.DataFrame(data, columns=['filename', 'rmse_distance'])
df_results['rank'] = df_results['rmse_distance'].rank(method='first', ascending=True).astype(int)
df_results = df_results[['rank', 'filename', 'rmse_distance']]
df_results = df_results.sort_values(by='rank')
df_results.to_csv('dairy_network_ranking_2009vs2016.csv', index=False)

,rank,filename,rmse_distance
211,1,data/USAMMv3_cattle_networks/dairy/dairy_netwo...,0.048176
382,2,data/USAMMv3_cattle_networks/dairy/dairy_netwo...,0.048355
432,3,data/USAMMv3_cattle_networks/dairy/dairy_netwo...,0.048440
938,4,data/USAMMv3_cattle_networks/dairy/dairy_netwo...,0.048506
745,5,data/USAMMv3_cattle_networks/dairy/dairy_netwo...,0.048537
...,...,...,...
694,996,data/USAMMv3_cattle_networks/dairy/dairy_netwo...,0.050803
711,997,data/USAMMv3_cattle_networks/dairy/dairy_netwo...,0.050822
991,998,data/USAMMv3_cattle_networks/dairy/dairy_netwo...,0.050881
257,999,data/USAMMv3_cattle_networks/dairy/dairy_netwo...,0.050885


In [29]:
df_2009 = pd.read_csv("data/USAMMv3_cattle_networks/dairy/dairy_network_20.network", sep = "\t")
df_2009.rename(columns = {"oStateAbbr": "org_st", "dStateAbbr": "dest_st"}, inplace=True)
df_2009

,oCountyId,dCountyId,dayOfYear,volume,commodity,period,org_st,dest_st,oPremId,dPremId,oPty,oBinnedSize,dPty,dBinnedSize
0,17133,42085,48,4,d,Q1,IL,PA,749132,770296,Frm,1200,Frm,350
1,17157,17027,82,5,d,Q1,IL,IL,751207,751640,Frm,1200,Frm,1200
2,17163,36115,69,3,d,Q1,IL,NY,749857,785038,Frm,350,Frm,1200
3,27139,27037,68,6,d,Q1,MN,MN,738979,739069,Frm,350,Frm,1200
4,27139,27045,9,50,d,Q1,MN,MN,738836,741234,Frm,350,Frm,1200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72361,36063,42105,298,2,d,Q4,NY,PA,812429,774053,Fdl,35,Frm,150
72362,36065,9005,348,1,d,Q4,NY,CT,814106,785545,Fdl,10,Frm,75
72363,36065,36045,321,1,d,Q4,NY,NY,814087,780637,Fdl,10,Frm,1200
72364,36065,42013,351,1,d,Q4,NY,PA,814090,773219,Fdl,35,Frm,75


In [30]:
df_2009['nb_shp'] = 1
df_2009

,oCountyId,dCountyId,dayOfYear,volume,commodity,period,org_st,dest_st,oPremId,dPremId,oPty,oBinnedSize,dPty,dBinnedSize,nb_shp
0,17133,42085,48,4,d,Q1,IL,PA,749132,770296,Frm,1200,Frm,350,1
1,17157,17027,82,5,d,Q1,IL,IL,751207,751640,Frm,1200,Frm,1200,1
2,17163,36115,69,3,d,Q1,IL,NY,749857,785038,Frm,350,Frm,1200,1
3,27139,27037,68,6,d,Q1,MN,MN,738979,739069,Frm,350,Frm,1200,1
4,27139,27045,9,50,d,Q1,MN,MN,738836,741234,Frm,350,Frm,1200,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72361,36063,42105,298,2,d,Q4,NY,PA,812429,774053,Fdl,35,Frm,150,1
72362,36065,9005,348,1,d,Q4,NY,CT,814106,785545,Fdl,10,Frm,75,1
72363,36065,36045,321,1,d,Q4,NY,NY,814087,780637,Fdl,10,Frm,1200,1
72364,36065,42013,351,1,d,Q4,NY,PA,814090,773219,Fdl,35,Frm,75,1


In [31]:
df_2009 = df_2009.groupby(['org_st', 'dest_st'])['nb_shp'].sum().reset_index()
df_2009.sort_values(by='nb_shp', ascending=False)

,org_st,dest_st,nb_shp
83,CA,CA,13744
275,IA,IA,7300
629,MN,MN,3718
959,NY,NY,3640
1459,WI,WI,3230
...,...,...,...
524,MD,CT,1
1091,PA,ND,1
1089,PA,MS,1
529,MD,IL,1


In [32]:
df_2009 = df_2009.loc[
    df_2009.org_st != df_2009.dest_st
]
df_2009.sort_values(by='nb_shp', ascending=False).head(20)

,org_st,dest_st,nb_shp
283,IA,MN,2340
963,NY,PA,1010
300,IA,WI,805
619,MN,IA,696
648,MN,WI,639
1438,WI,MN,537
971,NY,VT,467
1427,WI,IA,460
902,NM,TX,456
947,NY,MI,382


In [33]:
df_2009['nb_shp_norm_2009'] = (df_2009['nb_shp'] - df_2009['nb_shp'].min()) / (df_2009['nb_shp'].max() - df_2009['nb_shp'].min())
df_2009.sort_values(by='nb_shp_norm_2009', ascending=False)

,org_st,dest_st,nb_shp,nb_shp_norm_2009
283,IA,MN,2340,1.000000
963,NY,PA,1010,0.431381
300,IA,WI,805,0.343737
619,MN,IA,696,0.297136
648,MN,WI,639,0.272766
...,...,...,...,...
1050,OR,FL,1,0.000000
472,KY,ND,1,0.000000
1043,OK,VT,1,0.000000
1038,OK,SC,1,0.000000


In [34]:
df_merged = pd.merge(df_2009, df_2016, on=['org_st', 'dest_st'], how='outer')
df_merged

,org_st,dest_st,nb_shp_x,nb_shp_norm_2009,nb_shp_y,nb_shp_norm_2016
0,AL,CA,1.0,0.000000,NaN,NaN
1,AL,CO,1.0,0.000000,3.0,0.000638
2,AL,FL,9.0,0.003420,3.0,0.000638
3,AL,GA,6.0,0.002138,NaN,NaN
4,AL,IA,1.0,0.000000,3.0,0.000638
...,...,...,...,...,...,...
1639,WY,SD,3.0,0.000855,2.0,0.000319
1640,WY,TX,9.0,0.003420,1.0,0.000000
1641,WY,UT,2.0,0.000428,2.0,0.000319
1642,WY,WA,2.0,0.000428,NaN,NaN


In [35]:
# Fill NaN values in nb_shp_normalized_* columns with 0
df_merged[['nb_shp_norm_2016', 'nb_shp_norm_2009']] = df_merged[
    ['nb_shp_norm_2016', 'nb_shp_norm_2009']
].fillna(0)


In [36]:
df_merged = df_merged[['org_st', 'dest_st', 'nb_shp_norm_2009', 'nb_shp_norm_2016']]
df_merged.sort_values(by='nb_shp_norm_2016', ascending=False)

,org_st,dest_st,nb_shp_norm_2009,nb_shp_norm_2016
122,CA,TX,0.049594,1.000000
52,AZ,CA,0.029500,0.743622
1559,WI,IL,0.085934,0.419324
1001,NM,TX,0.194528,0.376276
1557,WI,IA,0.196238,0.341837
...,...,...,...,...
794,MS,WA,0.000000,0.000000
795,MS,WI,0.001710,0.000000
796,MT,AL,0.000000,0.000000
797,MT,AZ,0.000000,0.000000


In [37]:
rmse = np.sqrt(np.mean((df_merged['nb_shp_norm_2016'] - df_merged['nb_shp_norm_2009'])**2))
rmse

0.04890724309040452